In [57]:
import pandas as pd
import numpy as np

In [58]:
FILEPATH = "Y:/Members_Current/Jenn/EEG study/Imported data/cleaned/"
participants = ["105", "106", "107", "904", "905", "906"]

dfs = [pd.read_csv("%s%s_cleaned_ml.csv" % (FILEPATH, s)) for s in participants]

df = pd.concat(dfs, axis=0, ignore_index=True)

In [59]:
# samples are df rows mod 1200, pre is 0-199, post is 200-1199

# extract pre rows and post rows, randomly choose 200-400ms as post
pre = df[df.Time < 0]
post = df[df.Time >= 200][df.Time < 400]

c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [60]:
pre = pre.drop(columns=["Time", "E65"], axis=1)
post = post.drop(columns=["Time", "E65"], axis=1)

In [61]:
timeFrame = 200

preX = pre.values
(i,j) = preX.shape
preX = np.reshape(preX, (i // 200, j * 200))

In [62]:
postX = post.values
(i,j) = postX.shape
postX = np.reshape(postX, (i // 200, j * 200))

In [63]:
(preN, preD) = preX.shape
(postN, postD) = postX.shape

In [64]:
X = np.concatenate((preX, postX), axis=0)
(n, d) = X.shape
assert(n == preN + postN)
assert(d == postD)
assert(d == preD)

In [65]:
y = np.concatenate((np.zeros(preN), np.ones(postN)))
assert(y.shape == (n,))

In [66]:
# just use this : on next one
# from sklearn.model_selection import train_test_split 

rand_indices = np.random.permutation(n)
split = int(n * 0.8)
train_idx, valid_idx = rand_indices[:split], rand_indices[split:]
train_X, valid_X, train_y, valid_y = X[train_idx, :], X[valid_idx, :], y[train_idx], y[valid_idx]

In [67]:
# from sklearn.svm import SVC
# model = SVC(gamma='auto', kernel='linear')
# model.fit(train_X, train_y)

In [87]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(train_X, train_y)

c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [88]:
# from sklearn.neighbors import KNeighborsClassifier
# from dtw import DTW

# model = KNeighborsClassifier(metric=DTW)
# model.fit(train_X, train_y)

In [89]:
Xhat = model.predict(valid_X)

In [90]:
np.mean(Xhat != valid_y)

0.07734806629834254

In [91]:
Xhat

array([1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1.,
       1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1.])

In [92]:
valid_y

array([1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1.,
       1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.])

In [93]:
np.mean(model.predict(train_X) != train_y)

0.0027662517289073307

In [94]:
Xhat != valid_y

array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True, False, False, False, False, False, False,  True,
       False, False, False, False,  True, False, False, False, False,
       False, False,

In [95]:
from sklearn.metrics import classification_report
print(classification_report(valid_y, Xhat))

              precision    recall  f1-score   support

         0.0       0.90      0.96      0.93        98
         1.0       0.95      0.88      0.91        83

    accuracy                           0.92       181
   macro avg       0.93      0.92      0.92       181
weighted avg       0.92      0.92      0.92       181

